In [ ]:
%pip install faiss-cpu
%pip install langchain langchain_core langchain_community openai sentence_transformers

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
from openai import OpenAI

# Inicializa o cliente OpenAI com a URL base e a chave de API
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="sk-##########################", # Mude a chave de API para a sua chave OpenRouter
)

In [ ]:
# Carregando Embeddings locais com Sentence Transformers

def load_csv_data():
    # Carrega os documentos do arquivo CSV
    # Certifique-se de que o arquivo 'knowledge_base.csv' está no mesmo diretório do script
    loader = CSVLoader(file_path = "knowledge_base.csv")
    documents = loader.load()

    # Cria embeddings usando o modelo HuggingFace
    model_name = "all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)

    # Cria o vetor de armazenamento FAISS a partir dos documentos e embeddings
    vectorstore = FAISS.from_documents(documents, embeddings)
    retriever = vectorstore.as_retriever()
    return retriever

# Carrega os dados do CSV e cria o retriever
retriever = load_csv_data()

In [ ]:
# Loop de interação com o usuário
print("Bem-vindo ao Oraculo E.V.I.RAG!")
print("Digite 'sair' para encerrar a conversa.\n")

while True:
    # Solicita entrada do usuário
    user_input = input("Você: ")

    # Verifica se o usuário deseja sair
    if user_input.lower() == 'sair':
        print("Encerrando a conversa. Até logo!")
        break

    # Obtém documentos relevantes usando o retriever
    docs = retriever.get_relevant_documents(user_input)
    contexto = "\n".join([doc.page_content for doc in docs])

    # Cria o prompt para o modelo
    prompt = (
        "Você é um atendente de uma empresa. "
        "Seu trabalho é conversar com os clientes, consultando a base de conhecimentos da empresa, e dar uma resposta simples e precisa para ele, baseada na base de dados da empresa fornecida como contexto.\n\n"
        "Contexto:\n" + contexto # Inclui o contexto na mensagem de prompt
    )

    # Chamada API DeepSeek/OpenRouter
    response = client.chat.completions.create(
        model="deepseek/deepseek-chat-v3-0324:free",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_input}
        ],
        max_tokens=300 # Limite de tokens para a resposta (ajuste conforme necessário)
    )

    # Exibe a resposta do modelo
    if response:
        print("\nUsuário:", user_input)
        print("\nOraculo E.V.I.:", response.choices[0].message.content.strip())
    else:
        print("\n❌ Nenhuma resposta da API.")